TẢI THÊM ẢNH TỪ INTERNET XUỐNG VỚI NHỮNG NGƯỜI NỔI TIẾNG CÓ ẢNH DƯỚI 15 ẢNH

In [ ]:
def get_image(_id, description, name, article):
    import os
    from icrawler.builtin import GoogleImageCrawler

    # tạo thư mục với tên là _id
    dir_path = os.path.join('../data', 'images_crawl_more', str(_id) + " - " + name)
    os.makedirs(dir_path, exist_ok=True)

    # crawl images và lưu vào thư mục trên
    google_crawler = GoogleImageCrawler(
        feeder_threads=1,
        parser_threads=1,
        downloader_threads=8,
        storage={'root_dir': dir_path}
    )
    filters = dict(
        type='face',
        color='color'
    )
    try:
        google_crawler.crawl(
            keyword=description + ' ' + name + ' ' + article,
            filters=filters,
            max_num=200,
            min_size=(50, 50),
            file_idx_offset='auto'
        )
    except Exception as e:
        print(f"Error while crawling images for {name}: {e}")


In [ ]:
import os

# tìm thu muc chứa ít hơn 15 ảnh
image_path = "../data/images"
id_list = []
for folder in os.listdir(image_path):
    folder_path = os.path.join(image_path, folder)
    count = 0
    for image in os.listdir(folder_path):
        count += 1
    if count < 15:
        id_list.append(folder)

In [ ]:
print(id_list)

In [ ]:
import json
from tqdm import tqdm

# load data từ json
with open('../data/json/vietnam_celeb.json', 'r', encoding='utf-8') as f:
    celeb = json.load(f)

# Lặp qua danh sách các id trong id_list
for id in tqdm(id_list):
    # Tìm phần tử có id tương ứng trong danh sách celeb
    celeb_info = next((item for item in celeb if item["id"] == id), None)
    if celeb_info is not None:
        # Nếu tìm thấy phần tử, in ra thông tin "name", "description", "article"
        name = celeb_info.get("name", "")
        description = celeb_info.get("description", "")
        article = celeb_info.get("article", "")

        # Thực hiện lấy ảnh từ google image
        get_image(id, description, name, article)

EXTRACT FACE

In [ ]:
import os
import cv2
from deepface import DeepFace
from tqdm import tqdm
import time
import re


def extract_faces(img_path, save_path, file_name):
    results = DeepFace.extract_faces(
        img_path=img_path,
        target_size=(224, 224),
        detector_backend='mtcnn',
        enforce_detection=False,
        align=True
    )
    if len(results) > 0:
        if results[0]['confidence'] > 0:
            face = results[0]['face']
            face = (face * 255).astype('uint8')
            rgb_image = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            os.makedirs(save_path, exist_ok=True)
            cv2.imwrite(os.path.join(save_path, file_name.split(".")[0] + ".jpg"), rgb_image)


def number_label(folder_name):
    match = re.search(r'\d+', folder_name)  # Tìm kiếm một chuỗi các số liên tiếp
    if match:
        number = int(match.group())  # Chuyển chuỗi số thành số nguyên
        return number
    return 0


# Thư mục chứa ảnh
image_dir = "../data/images_crawl"
face_dir = "../data/images_face"

star_time = time.time()
# duyệt qua cac thu mục con
for folder in tqdm(os.listdir(image_dir), position=0):
    number = number_label(folder)
    if number > 0:  # tuỳ chỉnh đề tiếp tục khi gặp lỗi
        folder_path = os.path.join(image_dir, folder)
        face_path = os.path.join(face_dir, folder)
        for image in tqdm(os.listdir(folder_path), position=1, leave=False):
            image_path = os.path.join(folder_path, image)
            extract_faces(image_path, face_path, image)
end_time = time.time()
print("Đã làm xong! ", "- Thời gian:", round(end_time - star_time, 3))

In [ ]:
import numpy as np


def load_data(train_path, test_path):
    # load file train
    npzfile = np.load(train_path)
    x_train = npzfile['embedding']
    y_train = npzfile['label']

    #load file test
    npzfile = np.load(test_path)
    x_test = npzfile['embedding']
    y_test = npzfile['label']

    #show info
    print("X_train:", len(x_train), "y_train:", len(y_train))
    print("X_test:", len(x_test), "y_test:", len(y_test))
    print("Shape:", x_train[0].shape)

    return x_train, y_train, x_test, y_test

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


def knn(x_train, y_train, x_test, y_test):
    k_neighbors = [1, 3, 5, 7, 9, 11]
    for k in k_neighbors:
        classifier = KNeighborsClassifier(metric="cosine", n_neighbors=3)
        classifier.fit(x_train, y_train)
        y_pred = classifier.predict(x_test)

        # Tính toán các độ đo hiệu suất
        acc = accuracy_score(y_test, y_pred)
        prec = precision_score(y_test, y_pred, average='weighted', zero_division=1)
        rec = recall_score(y_test, y_pred, average='weighted', zero_division=1)
        f1 = f1_score(y_test, y_pred, average='weighted')

        print(f"k = {k}:")
        print(f"  Accuracy: {acc:.4f}")
        print(f"  Precision: {prec:.4f}")
        print(f"  Recall: {rec:.4f}")
        print(f"  F1-score: {f1:.4f}")

In [ ]:
#Facenet
train_path = "../data/embeddings/Facenet/train_facenet_embeddings.npz"
test_path = "../data/embeddings/Facenet/test_facenet_embeddings.npz"

x_train, y_train, x_test, y_test = load_data(train_path, test_path)
knn(x_train, y_train, x_test, y_test)

In [ ]:
#Facenet512
train_path = "../data/embeddings/Facenet512/train_Facenet512_embeddings.npz"
test_path = "../data/embeddings/Facenet512/test_Facenet512_embeddings.npz"

x_train, y_train, x_test, y_test = load_data(train_path, test_path)
knn(x_train, y_train, x_test, y_test)

In [ ]:
#VGG-Face
train_path = "../data/embeddings/VGG-Face/train_VGG-Face_embeddings.npz"
test_path = "../data/embeddings/VGG-Face/test_VGG-Face_embeddings.npz"

x_train, y_train, x_test, y_test = load_data(train_path, test_path)
knn(x_train, y_train, x_test, y_test)

In [ ]:
#ArcFace
train_path = "../data/embeddings/ArcFace/train_ArcFace_embeddings.npz"
test_path = "../data/embeddings/ArcFace/test_ArcFace_embeddings.npz"

x_train, y_train, x_test, y_test = load_data(train_path, test_path)
knn(x_train, y_train, x_test, y_test)

In [ ]:
#SFace
train_path = "../data/embeddings/SFace/train_SFace_embeddings.npz"
test_path = "../data/embeddings/SFace/test_SFace_embeddings.npz"

x_train, y_train, x_test, y_test = load_data(train_path, test_path)
knn(x_train, y_train, x_test, y_test)